# Track file analysis

## Volume tracks data format
 There are the following branches in the tree:

* npl - total number of plates passed by track
* nseg - total number of segments
* n0 - number of "holes" - plates where the segments were not found. In principle n0 should always be npl-nseg
* trid - id of the track
* t - the track itself (one EdbSegP object)
* s - array of measured segments (array of EdbSegP objects)
* sf - array of fitted segments (array of EdbSegP objects) 

The track t represents the track parameters on the one side of the long track line. 

The track line generally speaking is not straight and can not be represented as one vector only, but the array of vectors. In fact sf is the "fit function" of this track calculated according to Kalman Filter algorithm. 

For each measured (real) segment s[i] corresponds the fitted one sf[i] i.e. the best track estimation at the given Z. The t coincides with the first of with the last sf segment depending on the direction of the last fit. For the correct track fitting each segment s[i] should have coordinates, angles and the covariance matrix of errors set correctly. 

## Inspect quality of track reconstruction
First, we can access nseg and npl to inspect the length of the reconstructed tracks

In [1]:
import ROOT as r
import fedrarootlogon
import os

Welcome to JupyROOT 6.28/06
Load FEDRA libs


In [2]:
%jsroot on

In [3]:
c = r.TCanvas()
filepath="/eos/experiment/sndlhc/emulsionData/2022/CERN/emu_reco/RUN1/b000044/trackfiles/rootfiles/57_1/cell_9_9/linked_tracks.root"
inputfile = r.TFile.Open(filepath)
tracks = inputfile.Get("tracks")
tracks.SetLineColor(r.kGreen)
tracks.Draw("nseg","","")
tracks.SetLineColor(r.kBlack)
tracks.Draw("npl","","same")

#just adding some text on the canvas
t4 = r.TPaveText(.4,.75,.6,.85,"NBNDC")
t4.AddText("npl")  
t4.GetListOfLines().Last().SetTextColor(r.kBlack)
t4.AddText("nseg") 
t4.GetListOfLines().Last().SetTextColor(r.kGreen)
t4.SetFillColor(r.kWhite)
t4.Draw()

c.Draw()
    

We can see that npl has a peak at the maximum number of films (57), while nseg is quite lower, and not as regular.

This means that there are holes, i.e., films with missing segments. 

We can ask where the tracks start and end

In [4]:
cplate = r.TCanvas()
tracks.SetLineColor(r.kBlue)
tracks.Draw("s[nseg-1].Plate()>>hplateend(61,0,61)");
tracks.SetLineColor(r.kRed)
tracks.Draw("s[0].Plate()>>hplatestart(61,0,61)","","same");

#again, adding the legend
t1 = r.TPaveText(.6,.6,.8,.8,"NBNDC");
t1.AddText("First segment")
t1.GetListOfLines().Last().SetTextColor(r.kRed);
t1.AddText("Last segment") 
t1.GetListOfLines().Last().SetTextColor(r.kBlue);
t1.SetFillColor(r.kWhite);
t1.Draw();
cplate.Draw()

Finally, we can inspect if some film is more inefficient than the others: i.e., it has less segments than the others.
We can use a subsample of long tracks for this measurement: i.e., tracks with nseg>40

In [5]:
longtracks = "nseg>40"
cplates = r.TCanvas()
tracks.Draw("s.Plate()>>hplate(61,0,61)",longtracks)
r.gROOT.FindObject("hplate").SetTitle(";Plate")
cplates.Draw()

## Example of tracks tree reading with standard loop

Until now, we have used simple TTree->Draw functions to access the TTree contents.

Low-level analysis requires to access the single objects, so we will now show an example with a simple TTree.

Volume-tracks are in FEDRA represented as **EdbTrackP** objects, while base-tracks as **EdbSegP** objects.

We can ask directly python which functions we have available

In [6]:
#initializing empty reference system
dproc = r.EdbDataProc()
ali = r.EdbPVRec()
scancond = r.EdbScanCond()
ali.SetScanCond(scancond)

In [7]:
dproc.ReadTracksTree(ali,filepath,"npl==57") #just a few tracks for example

75724

EdbDataProc::ReadTracksTree: select 75724 of 517730 tracks by cut npl==57
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 56: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 55: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 54: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 53: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 52: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 51: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 50: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 47: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 46: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 45: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no pattern with pid 44: creating new one!
EdbDataProc::ReadTracksTree: WARNING: no patte

In [8]:
htx = r.TH1D("htx","TX of tracks;TX", 75,1.5,1.5)
hsz = r.TH1D("hsz","Z of segment positions;z[#mum]",100,-100000,1000)

## Starting loop

We do a loop on tracks, and a loop on segments from the track

In [9]:
for track in ali.eTracks:
    htx.Fill(track.TX())
    for iseg in range(track.N()):
        segment = track.GetSegment(iseg)
        hsz.Fill(segment.Z())

In [10]:
alltracks = ali.eTracks
len(alltracks)

75724

In [11]:
ctx = r.TCanvas("ctx","TX angle of tracks")
htx.DrawClone()
ctx.Draw()

In [12]:
csz = r.TCanvas("csz","Z position of track segments")
hsz.DrawClone()
csz.Draw()

In [13]:
whatisatrack = ali.eTracks[0]
help(whatisatrack)

Help on EdbTrackP in module ROOT._pythonization._titer object:

class EdbTrackP(EdbSegP)
 |  Method resolution order:
 |      EdbTrackP
 |      EdbSegP
 |      TObject
 |      EdbTrack2D
 |      EdbPoint2D
 |      EdbPoint
 |      EdbAngle2D
 |      CPPInstance
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  AddSegment(...)
 |      void EdbTrackP::AddSegment(EdbSegP* s)
 |  
 |  AddSegmentF(...)
 |      void EdbTrackP::AddSegmentF(EdbSegP* s)
 |  
 |  AddTrack(...)
 |      void EdbTrackP::AddTrack(const EdbTrackP& tr)
 |  
 |  AddVTA(...)
 |      void EdbTrackP::AddVTA(EdbVTA* vta)
 |  
 |  CHI2(...)
 |      float EdbTrackP::CHI2()
 |  
 |  CHI2F(...)
 |      float EdbTrackP::CHI2F()
 |  
 |  CheckAliasSegments(...)
 |      int EdbTrackP::CheckAliasSegments()
 |  
 |  CheckMaxGap(...)
 |      int EdbTrackP::CheckMaxGap()
 |  
 |  Class(...)
 |      static TClass* EdbTrackP::Class()
 |  
 |  Class_Name(...)
 |      static const char* EdbTrackP::Class_Name()
 |  
 |  Clas

In [14]:
whatisaseg = whatisatrack.GetSegment(0)
help(whatisaseg)

Help on EdbSegP in module ROOT._pythonization._titer object:

class EdbSegP(TObject, EdbTrack2D)
 |  Method resolution order:
 |      EdbSegP
 |      TObject
 |      EdbTrack2D
 |      EdbPoint2D
 |      EdbPoint
 |      EdbAngle2D
 |      CPPInstance
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Aid(...)
 |      int EdbSegP::Aid(int i)
 |  
 |  Angle(...)
 |      static float EdbSegP::Angle(const EdbSegP& s1, const EdbSegP& s2)
 |  
 |  COV(...)
 |      TMatrixT<double>& EdbSegP::COV()
 |  
 |  CheckCOV(...)
 |      bool EdbSegP::CheckCOV()
 |  
 |  Chi2(...)
 |      float EdbSegP::Chi2()
 |  
 |  Class(...)
 |      static TClass* EdbSegP::Class()
 |  
 |  Class_Name(...)
 |      static const char* EdbSegP::Class_Name()
 |  
 |  Class_Version(...)
 |      static short EdbSegP::Class_Version()
 |  
 |  Clear(...)
 |      void EdbSegP::Clear()
 |  
 |  Compare(...)
 |      int EdbSegP::Compare(const TObject* obj)
 |  
 |  Copy(...)
 |      void EdbSegP::Copy(const EdbS